In [ ]:
import pandas as pd
import numpy as np
import os
import re

from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoConfig
import torch

In [ ]:
# Carregar modelo e tokenizer
nome_modelo = "lfcc/bert-portuguese-ner"
tokenizer = AutoTokenizer.from_pretrained(nome_modelo)
modelo = AutoModelForTokenClassification.from_pretrained(nome_modelo)

# Função para anonimizar nomes no texto
def anonimizar_nome(texto):
    tamanho_pedido = 512

    # Tokenizar o texto e obter os offsets dos tokens
    entradas = tokenizer(
        texto,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=tamanho_pedido,
        return_offsets_mapping=True,  # Obter offsets dos tokens para processamento posterior
    )
    mapeamento_offset = entradas.pop("offset_mapping")[0]  # Remover do dicionário e armazenar separadamente

    # Executar o modelo para obter as previsões
    saídas = modelo(**entradas)
    logits = saídas.logits

    # Carregar a configuração do modelo para obter o mapeamento das labels
    configuracao = AutoConfig.from_pretrained(nome_modelo)
    mapeamento_labels = configuracao.id2label

    # Obter as previsões
    indices_classes_previstas = torch.argmax(logits, dim=-1)[0]
    labels_previstas = [mapeamento_labels[idx.item()] for idx in indices_classes_previstas]

    # Anonimizar o texto
    texto_anonimizado = list(texto)  # Converter para lista para substituição mutável

    for (inicio, fim), label in zip(mapeamento_offset, labels_previstas):
        if "Pessoa" in label:  # Verificar se o label está relacionado a uma pessoa
            for i in range(inicio, fim):  # Substituir o intervalo de caracteres no texto original
                texto_anonimizado[i] = "*"

    # Juntar a lista modificada de volta em uma string
    texto_anonimizado = "".join(texto_anonimizado)
    
    return texto_anonimizado

In [ ]:
# Função para anonimizar dados de arquivos Excel ou CSV
def anonimizar_arquivo(input_caminho, arquivo_saida, colunas_anonimizar):
    # Determinar o formato do arquivo pela extensão
    extensao_arquivo = os.path.splitext(input_caminho)[1].lower()
    
    # Ler o arquivo conforme seu formato
    if extensao_arquivo in ['.xlsx', '.xls']:
        df = pd.read_excel(input_caminho)  # Arquivo Excel
    elif extensao_arquivo == '.csv':
        df = pd.read_csv(input_caminho)  # Arquivo CSV
    else:
        raise ValueError("Formato de arquivo não suportado. Por favor, forneça um arquivo Excel (.xlsx, .xls) ou CSV (.csv).")

    # Garantir que 'colunas_anonimizar' seja uma lista, mesmo que seja uma string única
    if isinstance(colunas_anonimizar, str):
        colunas_anonimizar = [colunas_anonimizar]

    # Processar apenas as colunas que existem no DataFrame
    colunas_existentes = [col for col in colunas_anonimizar if col in df.columns]

    if not colunas_existentes:
        raise ValueError(f"Nenhuma das colunas especificadas existe no arquivo. Colunas disponíveis: {list(df.columns)}")

    # Remover linhas onde as colunas a serem anonimizadas estão vazias
    df = df.dropna(subset=colunas_existentes)

    for coluna in colunas_existentes:
        df[coluna] = df[coluna].apply(anonimizar_nome)  # Aplicar a anonimização nas colunas
    
    # Salvar o DataFrame atualizado no formato especificado
    extensao_saida = os.path.splitext(arquivo_saida)[1].lower()

    if extensao_saida in ['.xlsx', '.xls']:
        df.to_excel(arquivo_saida, index=False)
        print(f'Arquivo salvo como {arquivo_saida}')
    elif extensao_saida == '.csv':
        df.to_csv(arquivo_saida, index=False)
        print(f'Arquivo salvo como {arquivo_saida}')
    else:
        raise ValueError("Formato de arquivo de saída não suportado. Por favor, forneça um arquivo Excel (.xlsx, .xls) ou CSV (.csv).")

In [ ]:
# Informe o caminho do arquivo original, o nome do arquivo de saída e a(s) coluna(s) a serem processadas
anonimizar_arquivo('arquivo.xlsx', 'arquivo_anonimizado.xlsx', 'TEXTO_LIVRE')